In [ ]:
!pip install opencv-python
!pip install dlib
!pip install cmake
!pip install face_recognition

In [2]:
# Imoort libraries

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import math
import random
import time
import face_recognition
import cv2
import os

In [4]:
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from google.colab.patches import cv2_imshow

In [5]:
# Function to detect faces in images and printing the frame on the face

In [46]:
def detect_faces(path):
    # Check if path is a directory
    if os.path.isdir(path):
        # Process each file in the directory
        for image_name in os.listdir(path):
            image_path = os.path.join(path, image_name)
            process_image(image_path)
    elif os.path.isfile(path):
        # Process a single file
        process_image(path)
    else:
        print("The provided path is neither a directory nor a file.")

def process_image(image_path):
    # Ensure the path points to a file
    if os.path.isfile(image_path):
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image)

        image_cv = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        for top, right, bottom, left in face_locations:
            cv2.rectangle(image_cv, (left, top), (right, bottom), (0, 255, 0), 13)

        image_rgb = cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)
        plt.imshow(image_rgb)
        plt.axis('off')
        # Extract image name for the title
        image_name = os.path.basename(image_path)
        plt.title(f"Faces detected in {image_name}")
        plt.show()
    else:
        print(f"No valid file found at {image_path}")

In [ ]:
detect_faces('/content/drive/MyDrive/Path/To/Your/Dataset/Folders')

In [ ]:
# Extract Features and Save with Person's Name

In [7]:
def save_encodings(encodings_file, encodings):
    with open(encodings_file, 'wb') as file:
        pickle.dump(encodings, file)

In [8]:
def load_encodings(encodings_file):
    try:
        if Path(encodings_file).exists():
            with open(encodings_file, 'rb') as file:
                return pickle.load(file)
    except EOFError:
        pass
    return {}

In [18]:
def add_person(person_folder_path, encodings_file='face_encodings.pkl'):
    person_name = os.path.basename(person_folder_path)
    known_faces = load_encodings(encodings_file)

    existing_encodings = known_faces.get(person_name, [])
    initial_encoding_count = len(existing_encodings)
    new_encodings_count = 0

    for image_name in os.listdir(person_folder_path):
        image_path = os.path.join(person_folder_path, image_name)
        image = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(image)
        if encodings:
            if all(not np.array_equal(encodings[0], existing_encoding) for existing_encoding in existing_encodings):
                existing_encodings.append(encodings[0])
                new_encodings_count += 1

    if new_encodings_count > 0:
        known_faces[person_name] = existing_encodings
        save_encodings(encodings_file, known_faces)
        print(f"Updated {person_name} in the dataset with {new_encodings_count} new encodings.")
    elif initial_encoding_count > 0:
        print(f"No new encodings found for {person_name}.")
    else:
        print(f"No valid face encodings found for {person_name}. Not added to dataset.")

In [ ]:
add_person('/content/drive/MyDrive/Path/To/Your/Dataset/Folders')

In [ ]:
# Function To Recognize Faces

In [27]:
def recognize_person(image_path, encodings_file='face_encodings.pkl'):
    unknown_image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(unknown_image)
    unknown_encodings = face_recognition.face_encodings(unknown_image, face_locations)

    image_cv = cv2.cvtColor(unknown_image, cv2.COLOR_RGB2BGR)

    known_faces = {}
    with open(encodings_file, 'rb') as file:
        known_faces = pickle.load(file)

    recognized_names = []
    for (top, right, bottom, left), unknown_face_encoding in zip(face_locations, unknown_encodings):
        name = "Unknown"

        for known_name, known_encodings in known_faces.items():
            matches = face_recognition.compare_faces(known_encodings, unknown_face_encoding)
            if True in matches:
                name = known_name
                recognized_names.append(name)
                break

        cv2.rectangle(image_cv, (left, top), (right, bottom), (0, 255, 0), 2)

        label_background_color = (50, 205, 50)
        text_color = (0, 0, 0)

        cv2.rectangle(image_cv, (left, bottom - 5), (right, bottom + 35), label_background_color, cv2.FILLED)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image_cv, name, (left + 6, bottom + 30), font, 1.6, text_color, 2)
    print("Recognized persons:", ", ".join(recognized_names))

    scale_percent = 30
    width = int(image_cv.shape[1] * scale_percent / 100)
    height = int(image_cv.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(image_cv, dim, interpolation=cv2.INTER_AREA)

    cv2_imshow(resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
recognize_person('/content/drive/MyDrive/Path/To/Your/Dataset/Folders/New_Images')

In [ ]:
# Function to create dataframe for the saved encodings

In [11]:
def create_dataset_dataframe(encodings_file='face_encodings.pkl'):
    dataset = []
    id_counter = 0

    with open(encodings_file, 'rb') as file:
        while True:
            try:
                known_faces = pickle.load(file)
                for name, encodings in known_faces.items():
                    dataset.append({
                        'Name': name,
                        'ID': id_counter,
                        'Number of Images': len(encodings)
                    })
                    id_counter += 1
            except EOFError:
                break

    df = pd.DataFrame(dataset)
    print(df)

In [ ]:
create_dataset_dataframe()

In [49]:
# Summary of the functions

In [ ]:
detect_faces() # used to only detect faces (it accept parameters as image path or folder path containing images in it)
add_person() # Used to add a new person to the dataset to be recognized later
recognize_person() # Used to recognize persons that already been added to the dataset
create_dataset_dataframe() # Used to return the dataset containing how many images for each person, person's ID and Persons Name